In [1]:
import pandas as pd

In [4]:
# Load the provided Excel files
order_report_path = 'Company X - Order Report.xlsx'
pincode_zones_path = 'Company X - Pincode Zones.xlsx'
sku_master_path = 'Company X - SKU Master.xlsx'
courier_invoice_path = 'Courier Company - Invoice.xlsx'
courier_rates_path = 'Courier Company - Rates.xlsx'

In [5]:
# Reading the data from each file
order_report = pd.read_excel(order_report_path)
pincode_zones = pd.read_excel(pincode_zones_path)
sku_master = pd.read_excel(sku_master_path)
courier_invoice = pd.read_excel(courier_invoice_path)
courier_rates = pd.read_excel(courier_rates_path)

In [7]:
dfs = {
    'Order Report': order_report,
    'Pincode Zones': pincode_zones,
    'SKU Master': sku_master,
    'Courier Invoice': courier_invoice,
    'Courier Rates': courier_rates
}

df_overview = {name: df.head() for name, df in dfs.items()}
df_overview

{'Order Report':    ExternOrderNo            SKU  Order Qty
 0     2001827036  8904223818706        1.0
 1     2001827036  8904223819093        1.0
 2     2001827036  8904223819109        1.0
 3     2001827036  8904223818430        1.0
 4     2001827036  8904223819277        1.0,
 'Pincode Zones':    Warehouse Pincode  Customer Pincode Zone
 0             121003            507101    d
 1             121003            486886    d
 2             121003            532484    d
 3             121003            143001    b
 4             121003            515591    d,
 'SKU Master':              SKU  Weight (g)
 0  8904223815682         210
 1  8904223815859         165
 2  8904223815866         113
 3  8904223815873          65
 4  8904223816214         120,
 'Courier Invoice':         AWB Code    Order ID  Charged Weight  Warehouse Pincode  \
 0  1091117222124  2001806232            1.30             121003   
 1  1091117222194  2001806273            1.00             121003   
 2  109111722

In [8]:
# Merge the Order Report with SKU Master to get the weight of each SKU in an order
order_with_weight = pd.merge(order_report, sku_master, on='SKU', how='left')

In [9]:
# Calculate the total weight of each order by multiplying the SKU weight with order quantity and summing up per order
order_with_weight['Total Weight (g)'] = order_with_weight['Weight (g)'] * order_with_weight['Order Qty']
order_total_weight = order_with_weight.groupby('ExternOrderNo')['Total Weight (g)'].sum().reset_index()

In [10]:
# Rename columns for clarity
order_total_weight.rename(columns={'ExternOrderNo': 'Order ID', 'Total Weight (g)': 'Total Weight per X (g)'}, inplace=True)

In [11]:
# Convert weight from grams to kilograms as that is the unit used in courier data
order_total_weight['Total Weight per X (KG)'] = order_total_weight['Total Weight per X (g)'] / 1000

In [12]:
# Now we need to merge the total weight per order with the pincode zones data to get the delivery zones
# First, we need to ensure the Order ID column is of the same type in both the order report and courier invoice
order_total_weight['Order ID'] = order_total_weight['Order ID'].astype(str)
courier_invoice['Order ID'] = courier_invoice['Order ID'].astype(str)

In [13]:
# Merge the total weight per order with the courier invoice to get the warehouse and customer pincodes
order_weight_with_zones = pd.merge(order_total_weight, courier_invoice[['Order ID', 'Warehouse Pincode', 'Customer Pincode']], on='Order ID', how='left')

In [14]:
# Then, we merge with the pincode zones data to get the delivery zone
order_weight_with_zones = pd.merge(order_weight_with_zones, pincode_zones, on=['Warehouse Pincode', 'Customer Pincode'], how='left')

In [15]:
# Now we have the total weight and delivery zone per order, we can calculate the expected charges
order_weight_with_zones.head()

,Order ID,Total Weight per X (g),Total Weight per X (KG),Warehouse Pincode,Customer Pincode,Zone
0,2001806210,220.0,0.220,121003,140604,b
1,2001806226,480.0,0.480,121003,723146,d
2,2001806229,500.0,0.500,121003,421204,d
3,2001806232,1302.0,1.302,121003,507101,d
4,2001806233,245.0,0.245,121003,263139,b


In [17]:
# Define a function to calculate the expected charges based on the weight and zone
def calculate_expected_charges(weight, zone, rates):
    # Find the rate information for the given zone
    rate_info = rates[rates['Zone'] == zone.upper()].iloc[0]
    
    # Calculate the fixed charge based on the first weight slab
    fixed_charge = rate_info['Forward Fixed Charge']
    
    # Calculate additional charge for weight above the first slab
    additional_weight = max(weight - rate_info['Weight Slabs'], 0)
    additional_charge = (additional_weight // rate_info['Weight Slabs']) * rate_info['Forward Additional Weight Slab Charge']
    
    # Total expected charge is the sum of the fixed and additional charges
    total_expected_charge = fixed_charge + additional_charge
    
    return total_expected_charge

In [21]:
# Apply the function to each order in the dataframe to calculate the expected charges
order_weight_with_zones['Expected Charge (Rs.)'] = order_weight_with_zones.apply(
    lambda row: calculate_expected_charges(row, courier_rates),
    axis=1
)

In [22]:
# Merge the expected charges with the courier invoice data to compare with billed charges
comparison_df = pd.merge(
    order_weight_with_zones[['Order ID', 'Expected Charge (Rs.)']],
    courier_invoice[['Order ID', 'Billing Amount (Rs.)']],
    on='Order ID',
    how='left'
)

In [23]:
# Add a column to determine if the charge is correct, over, or under
comparison_df['Charge Discrepancy (Rs.)'] = comparison_df['Billing Amount (Rs.)'] - comparison_df['Expected Charge (Rs.)']

In [24]:
# Now let's view the first few rows to verify the calculations
comparison_df.head()

,Order ID,Expected Charge (Rs.),Billing Amount (Rs.),Charge Discrepancy (Rs.)
0,2001806210,33.0,174.5,141.5
1,2001806226,45.4,90.2,44.8
2,2001806229,45.4,90.2,44.8
3,2001806232,45.4,135.0,89.6
4,2001806233,33.0,61.3,28.3
